<a href="https://colab.research.google.com/github/deer-in-haze/gilusis_mokymas/blob/main/Migle_Lukoseviciute_lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importai

In [28]:
import os
import ast
import re
import json
from getpass import getpass

import pandas as pd
import google.generativeai as genai

Gemini API key

In [29]:
os.environ["GEMINI_API_KEY"] = getpass("Enter your API key: ")
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

Modeliai

In [30]:
models_to_test = [
    "gemini-2.5-pro",
    "gemini-2.5-flash",
    "gemini-2.5-flash-lite",
    "gemma-3-1b-it",
    "gemma-3-4b-it",
    "gemma-3-27b-it"
]

Prisijungimo prie modelių testas

In [31]:
prompt = "Hello!"

for model_id in models_to_test:
    try:
        response = genai.GenerativeModel(model_id).generate_content(prompt)
        print(f"\n--- {model_id} ---")
        print(response.text)
    except Exception as e:
        print(f"\n--- {model_id} ---")
        print("Error:", e)


--- gemini-2.5-pro ---
Hello there! How can I help you today?

--- gemini-2.5-flash ---
Hello there! How can I help you today?

--- gemini-2.5-flash-lite ---
Hello there! How can I help you today?

--- gemma-3-1b-it ---
Hello there! How's your day going so far? 😊 

Is there anything you'd like to chat about, or anything I can help you with?

--- gemma-3-4b-it ---
Hello there! How’s your day going so far? Is there anything I can help you with today? 😊

--- gemma-3-27b-it ---
Hello there! 👋 

It's great to connect with you. How can I help you today? 

Just let me know what you're thinking, whether you have a question, want to brainstorm ideas, need some information, or just want to chat. I'm here and ready to assist! 😊






Užduotis

In [32]:
prompt = """
Your task is to solve the following challenge. Follow every step carefully. The final answer must be returned in exact JSON format:
{
  "fileReadSuccess": <boolean>,
  "textSplitUpperBoundaryFound": <boolean>,
  "textSplitBottomBoundaryFound": <boolean>,
  "number_of_times_ghost_appears": <number>,
  "number_of_times_shadow_appears": <number>,
  "number_of_times_widger_appears": <number>,
  "answer": <number>
}.
Use the file located at this URL:
https://www.gutenberg.org/cache/epub/98/pg98.txt
From that text ignore everything before the line that starts with:
*** START OF THE PROJECT GUTENBERG EBOOK A TALE OF TWO CITIES ***
and ignore everything after the line that starts with:
*** END OF THE PROJECT GUTENBERG EBOOK A TALE OF TWO CITIES ***
.
Within the cleaned text:
Count how many times the exact word "ghost" (case-insensitive) appears.
Count how many times the exact word "shadow" (case-insensitive) appears.
Count how many times the exact word "widger" (case-insensitive) appears.
Compute the following expression (answer):
(number_of_times_ghost_appears × 7) + (number_of_times_shadow_appears × 13) + number_of_times_widger_appears
Return only the result as JSON, no explanations.
"""
expectedJson = {
  "fileReadSuccess": True,
  "textSplitUpperBoundaryFound": True,
  "textSplitBottomBoundaryFound": True,
  "number_of_times_ghost_appears": 11,
  "number_of_times_shadow_appears": 25,
  "number_of_times_widger_appears": 0,
  "answer": 402
}

requiredKeys = list(expectedJson.keys())

Pagalbinės funkcijos

In [33]:
def _extract_structured_obj(text: str):
    if not text:
        return None, "empty response"

    s = text.strip()

    m = re.search(r"```(?:json|JSON)\s*\n(.*?)```", s, flags=re.DOTALL)
    if not m:
        m = re.search(r"```[\w+-]*\s*\n(.*?)```", s, flags=re.DOTALL)
    candidate = m.group(1).strip() if m else s

    try:
        obj = json.loads(candidate)
        if isinstance(obj, dict):
            return obj, None
    except Exception:
        pass

    m = re.search(r"\{.*?\}", candidate, flags=re.DOTALL)
    if m:
        block = m.group(0)
        try:
            obj = json.loads(block)
            if isinstance(obj, dict):
                return obj, None
        except Exception:
            try:
                obj = ast.literal_eval(block)
                if isinstance(obj, dict):
                    return obj, None
            except Exception:
                pass

    return None, "could not extract a JSON/Python dict payload"

def _normalize_for_compare(value, expected):

    if isinstance(expected, bool):
        if isinstance(value, str):
            v = value.strip().lower()
            if v in ("true", "false"):
                return v == "true"
        return bool(value) if isinstance(value, (bool, int)) else value
    if isinstance(expected, int):
        if isinstance(value, str) and re.fullmatch(r"-?\d+", value.strip()):
            return int(value.strip())
    if isinstance(expected, float):
        if isinstance(value, (int, float)):
            return float(value)
        if isinstance(value, str) and re.fullmatch(r"-?\d+(?:\.\d+)?", value.strip()):
            return float(value.strip())
    return value

def assess_model_response_flat(model_id, model_output, expected_data, required_keys=None):
    if required_keys is None:
        required_keys = list(expected_data.keys())

    obj, reason = _extract_structured_obj(model_output)
    row = {"model": model_id}

    if obj is None:
        row.update({k: None for k in required_keys})
        row["PASS"] = False
        row["reason"] = f"Could not extract JSON: {reason}"
        return row

    mismatches = []
    for k in required_keys:
        got_raw = obj.get(k, None)
        expected = expected_data[k]
        got_norm = _normalize_for_compare(got_raw, expected)
        match = (got_norm == expected)
        row[k] = got_raw
        if not match:
            mismatches.append(f"{k}: expected {expected!r}, got {got_raw!r}")

    row["PASS"] = len(mismatches) == 0
    row["reason"] = "; ".join(mismatches) if mismatches else "All match"
    return row

Generavimas su visais modeliais

In [34]:
rows = []

for model_id in models_to_test:
    try:
        response = genai.GenerativeModel(model_id).generate_content(prompt)
        raw = getattr(response, "text", "") or ""
        row = assess_model_response_flat(model_id, raw, expectedJson, required_keys=requiredKeys)
        rows.append(row)
    except Exception as e:
        rows.append({"model": model_id, "PASS": False, "reason": str(e)})

expected_row = {"model": "EXPECTED"}
for k, v in expectedJson.items():
    expected_row[k] = v
expected_row["PASS"] = True
expected_row["reason"] = "Ground truth"
rows.append(expected_row)

df = pd.DataFrame(rows)

df.to_csv("rezultatai.csv", index=False)